In [1]:
import os

In [2]:
%pwd

'd:\\Nikku\\End to End Project\\Kidney-Disease-Classification\\research'

In [3]:
os.chdir("..")

In [4]:
%pwd

'd:\\Nikku\\End to End Project\\Kidney-Disease-Classification'

In [5]:
# Update the entity

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [7]:
# Update the Configuration Manager
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories



class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [8]:
# Update the Components

import os
import gdown
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_data(self) -> str:
        '''
        Fetch data from Url
        '''
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok= True)
            logger.info(f"Downloading data from {dataset_url} inot file {zip_download_dir}")
            
            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        '''
        zip_file_path: str
        Extract the zip file into the data directory
        Funxtion returns None
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

In [9]:
# Update the pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-13 11:48:14,475: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-13 11:48:14,477: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-13 11:48:14,477: INFO: common: Created directory at: artifacts]
[2023-10-13 11:48:14,478: INFO: common: Created directory at: artifacts/data_ingestion]
[2023-10-13 11:48:14,479: INFO: 1799066668: Downloading data from https://drive.google.com/file/d/1NGlWlZmoEDjrNYMjc-9OgIwuebY68wDh/view?usp=sharing inot file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1NGlWlZmoEDjrNYMjc-9OgIwuebY68wDh
From (redirected): https://drive.google.com/uc?/export=download&id=1NGlWlZmoEDjrNYMjc-9OgIwuebY68wDh&confirm=t&uuid=a51c8048-f6cb-420f-a8eb-aae4f7170631
To: d:\Nikku\End to End Project\Kidney-Disease-Classification\artifacts\data_ingestion\data.zip
100%|██████████| 1.63G/1.63G [06:17<00:00, 4.30MB/s]

[2023-10-13 11:54:34,757: INFO: 1799066668: Downloaded data from https://drive.google.com/file/d/1NGlWlZmoEDjrNYMjc-9OgIwuebY68wDh/view?usp=sharing into file artifacts/data_ingestion/data.zip]
